In [1]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
%load_ext autoreload
%autoreload
from IPython.display import clear_output

import os
os.environ['PYTHONHASHSEED'] = '0'
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

import sys
import argparse
import subprocess
import pdb
import time
import random
import _pickle as cPickle
import matplotlib.pyplot as plt
import glob

%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf

from data_structure import get_batches
from hntm import HierarchicalNeuralTopicModel
from nhdp import nestedHierarchicalNeuralTopicModel
from tsgntm import TreeStructuredGaussianNeuralTopicModel
from tree import get_descendant_idxs
from evaluation import get_hierarchical_affinity, get_topic_specialization, print_topic_sample, print_topic_year
from configure import get_config

pd.set_option('display.max_columns', 30)

# load data & set config

In [3]:
config = get_config(nb_name)

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = config.gpu
np.random.seed(config.seed)
random.seed(config.seed)

In [5]:
instances_train, instances_dev, instances_test, word_to_idx, idx_to_word, bow_idxs = cPickle.load(open(config.path_data,'rb'))
train_batches = get_batches(instances_train, config.batch_size)
dev_batches = get_batches(instances_dev, config.batch_size)
test_batches = get_batches(instances_test, config.batch_size)
config.dim_bow = len(bow_idxs)

In [6]:
def debug(variable, sample_batch=None):
    if sample_batch is None: sample_batch = test_batches[0][1]
    feed_dict = model.get_feed_dict(sample_batch, mode='test')
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

def check(variable):
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    sample_batch = test_batches[0]
    feed_dict = model.get_feed_dict(sample_batch, mode='test', assertion=True)
    _variable = sess.run(variable, feed_dict=feed_dict)
    return _variable

# model

## initialize model

In [7]:
if 'sess' in globals(): sess.close()
TopicModels = {'hntm': HierarchicalNeuralTopicModel, 'nhdp': nestedHierarchicalNeuralTopicModel, 'tsgntm': TreeStructuredGaussianNeuralTopicModel}
TopicModel = TopicModels[config.model]
model = TopicModel(config)
sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(max_to_keep=config.max_to_keep)
update_tree_flg = False

# run

## initialize log

In [8]:
checkpoint = []
losses_train = []
ppls_train = []
ppl_min = np.inf
epoch = 0
train_batches = get_batches(instances_train, config.batch_size, iterator=True)

log_df = pd.DataFrame(columns=pd.MultiIndex.from_tuples(
                    list(zip(*[['','','','TRAIN:','','','','','','VALID:','','','','','','TEST:','', 'SPEC:', '', '', 'HIER:', ''],
                            ['Time','Ep','Ct','LOSS','PPL','NLL','KL', 'GAUSS', 'REG','LOSS','PPL','NLL','KL', 'GAUSS','REG','LOSS','PPL', '1', '2', '3', 'CHILD', 'OTHER']]))))

cmd_rm = 'rm -r %s' % config.dir_model
res = subprocess.call(cmd_rm.split())
cmd_mk = 'mkdir %s' % config.dir_model
res = subprocess.call(cmd_mk.split())

def update_checkpoint(config, checkpoint, global_step):
    checkpoint.append(config.path_model + '-%i' % global_step)
    if len(checkpoint) > config.max_to_keep:
        path_model = checkpoint.pop(0) + '.*'
        for p in glob.glob(path_model):
            os.remove(p)
    cPickle.dump(checkpoint, open(config.path_checkpoint, 'wb'))
    
def validate(sess, batches, model):
    losses = []
    ppl_list = []
    prob_topic_list = []
    n_bow_list = []
    n_topics_list = []
    for ct, batch in batches:
        feed_dict = model.get_feed_dict(batch, mode='test')
        loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, prob_topic_batch, n_bow_batch, n_topics_batch \
            = sess.run([model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, model.prob_topic, model.n_bow, model.n_topics], feed_dict = feed_dict)
        losses += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppl_list += list(ppls_batch)
        prob_topic_list.append(prob_topic_batch)
        n_bow_list.append(n_bow_batch)
        n_topics_list.append(n_topics_batch)
    loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean = np.mean(losses, 0)
    ppl_mean = np.exp(np.mean(ppl_list))
    
    probs_topic = np.concatenate(prob_topic_list, 0)
    
    n_bow = np.concatenate(n_bow_list, 0)
    n_topics = np.concatenate(n_topics_list, 0)
    probs_topic_mean = np.sum(n_topics, 0) / np.sum(n_bow)
    
    return loss_mean, topic_loss_recon_mean, topic_loss_kl_mean, topic_loss_gauss_mean, topic_loss_reg_mean, ppl_mean, probs_topic_mean    

## train & validate model

In [9]:
time_start = time.time()
while epoch < config.n_epochs:
    # train
    for ct, batch in train_batches:
        feed_dict = model.get_feed_dict(batch)
        _, loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch, ppls_batch, global_step_log = \
        sess.run([model.opt, model.loss, model.topic_loss_recon, model.topic_loss_kl, model.topic_loss_gauss, model.topic_loss_reg, model.topic_ppls, tf.train.get_global_step()], feed_dict = feed_dict)

        losses_train += [[loss_batch, topic_loss_recon_batch, topic_loss_kl_batch, topic_loss_gauss_batch, topic_loss_reg_batch]]
        ppls_train += list(ppls_batch)

#         if global_step_log % config.log_period == 0:
        if global_step_log % 1000 == 0:
            # validate
            loss_train, topic_loss_recon_train, topic_loss_kl_train, topic_loss_gauss_train, topic_loss_reg_train = np.mean(losses_train, 0)
            ppl_train = np.exp(np.mean(ppls_train))
            loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_gauss_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)

            # test
            if ppl_dev < ppl_min:
                ppl_min = ppl_dev
                loss_test, _, _, _, _, ppl_test, _ = validate(sess, test_batches, model)
                saver.save(sess, config.path_model, global_step=global_step_log)
                cPickle.dump(config, open(config.path_config % global_step_log, 'wb'))
                update_checkpoint(config, checkpoint, global_step_log)
            
            # visualize topic
            topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
            topics_freq_idxs = bow_idxs[topics_freq_indices]
            topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
            topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
            descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
            recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
            
            depth_specs = get_topic_specialization(sess, model, instances_test)
            hierarchical_affinities = get_hierarchical_affinity(sess, model)
            
            # log
            clear_output()
            time_log = int(time.time() - time_start)
            log_series = pd.Series([time_log, epoch, ct, \
                    '%.2f'%loss_train, '%.0f'%ppl_train, '%.2f'%topic_loss_recon_train, '%.2f'%topic_loss_kl_train, '%.2f'%topic_loss_gauss_train, '%.2f'%topic_loss_reg_train, \
                    '%.2f'%loss_dev, '%.0f'%ppl_dev, '%.2f'%topic_loss_recon_dev, '%.2f'%topic_loss_kl_dev, '%.2f'%topic_loss_gauss_dev, '%.2f'%topic_loss_reg_dev, \
                    '%.2f'%loss_test, '%.0f'%ppl_test, \
                    '%.2f'%depth_specs[1], '%.2f'%depth_specs[2], '%.2f'%depth_specs[3], \
                    '%.2f'%hierarchical_affinities[0], '%.2f'%hierarchical_affinities[1]],
                    index=log_df.columns)
            log_df.loc[global_step_log] = log_series
            display(log_df)
            cPickle.dump(log_df, open(os.path.join(config.path_log), 'wb'))
            print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)
            print(np.sum(debug(model.topic_logvars), 1))
            
            # update tree
            if not config.static:
                config.tree_idxs, update_tree_flg = model.update_tree(topic_prob_topic, recur_prob_topic)
                if update_tree_flg:
                    print(config.tree_idxs)
                    name_variables = {tensor.name: variable for tensor, variable in zip(tf.global_variables(), sess.run(tf.global_variables()))} # store paremeters
                    if 'sess' in globals(): sess.close()
                    model = HierarchicalNeuralTopicModel(config)
                    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1))
                    name_tensors = {tensor.name: tensor for tensor in tf.global_variables()}
                    sess.run([name_tensors[name].assign(variable) for name, variable in name_variables.items()]) # restore parameters
                    saver = tf.train.Saver(max_to_keep=1)
                
            time_start = time.time()

    train_batches = get_batches(instances_train, config.batch_size, iterator=True)
    epoch += 1

loss_dev, topic_loss_recon_dev, topic_loss_kl_dev, topic_loss_reg_dev, ppl_dev, probs_topic_dev = validate(sess, dev_batches, model)
topics_freq_indices = np.argsort(sess.run(model.topic_bow), 1)[:, ::-1][:, :config.n_freq]
topics_freq_idxs = bow_idxs[topics_freq_indices]
topic_freq_tokens = {topic_idx: [idx_to_word[idx] for idx in topic_freq_idxs] for topic_idx, topic_freq_idxs in zip(model.topic_idxs, topics_freq_idxs)}
topic_prob_topic = {topic_idx: prob_topic for topic_idx, prob_topic in zip(model.topic_idxs, probs_topic_dev)}
descendant_idxs = {parent_idx: get_descendant_idxs(model, parent_idx) for parent_idx in model.topic_idxs}
recur_prob_topic = {parent_idx: np.sum([topic_prob_topic[child_idx] for child_idx in recur_child_idxs]) for parent_idx, recur_child_idxs in descendant_idxs.items()}
display(log_df)
print_topic_sample(sess, model, topic_prob_topic=topic_prob_topic, recur_prob_topic=recur_prob_topic, topic_freq_tokens=topic_freq_tokens)

TRAIN:                                  VALID:        \
       Time  Ep    Ct    LOSS   PPL     NLL    KL GAUSS   REG    LOSS   PPL   
1000     85   0   999  928.91  2691  923.50  1.60  3.57  0.24  888.19  2519   
2000     71   0  1999  921.96  2538  916.06  2.13  3.53  0.24  880.70  2365   
3000     65   1   295  919.36  2468  913.31  2.42  3.39  0.24  878.76  2315   
4000     67   1  1295  916.82  2422  910.56  2.62  3.40  0.23  873.13  2193   
5000     66   1  2295  914.61  2385  908.19  2.77  3.43  0.23  872.92  2201   
...     ...  ..   ...     ...   ...     ...   ...   ...   ...     ...   ...   
170000   69  62  2351  895.10  1999  885.58  4.53  4.83  0.15  857.76  1905   
171000   69  63   647  895.10  1998  885.57  4.53  4.84  0.15  856.93  1891   
172000   67  63  1647  895.08  1998  885.54  4.54  4.84  0.15  856.88  1891   
173000   68  63  2647  895.06  1998  885.52  4.54  4.84  0.15  856.71  1886   
174000   69  64   943  895.04  1997  885.50  4.54  4.84  0.15  858.14  1919   

                                   TEST:       SPEC:             HIER:        
           NLL    KL GAUSS   REG    LOSS   PPL     1     2     3 CHILD OTHER  
1000    881.96  1.91  4.08  0.24  885.92  2467  0.25  0.22  0.20  0.94  0.86  
2000    874.29  3.13  2.93  0.35  879.34  2362  0.33  0.22  0.22  0.94  0.84  
3000    871.83  3.15  3.54  0.24  878.41  2325  0.17  0.23  0.24  0.93  0.81  
4000    865.86  3.58  3.50  0.19  877.21  2311  0.24  0.22  0.24  0.93  0.79  
5000    865.64  3.50  3.59  0.20  877.21  2311  0.18  0.21  0.25  0.93  0.84  
...        ...   ...   ...   ...     ...   ...   ...   ...   ...   ...   ...  
170000  847.83  4.70  5.05  0.17  859.45  1954  0.20  0.25  0.31  0.83  0.68  
171000  847.07  4.73  5.01  0.12  859.45  1954  0.22  0.24  0.30  0.85  0.69  
172000  846.96  4.73  5.05  0.13  859.45  1954  0.22  0.25  0.32  0.84  0.72  
173000  846.82  4.73  5.00  0.16  859.45  1954  0.21  0.22  0.31  0.85  0.72  
174000  848.33  4.69  5.00  0.12  859.45  1954  0.19  0.22  0.31  0.82  0.69  

[174 rows x 22 columns]

0 R: 1.000 P: 0.039 meat beef fried portions soup cooked fish steak waitress dishes
   1 R: 0.355 P: 0.024 meat fish dishes shrimp fried dish pork steak enjoyed fantastic
     11 R: 0.172 P: 0.172 rice thai mexican tacos chinese pho soup rolls spicy beef
     12 R: 0.061 P: 0.061 rice dishes meat shrimp dish portions pork fish flavor beef
     13 R: 0.098 P: 0.098 steak wine wonderful enjoyed fantastic cooked bread attentive husband italian
   2 R: 0.210 P: 0.007 coffee cream ice free options inside tea wonderful found kind
     21 R: 0.004 P: 0.004 sandwich coffee free options huge line bread stop open large
     22 R: 0.104 P: 0.104 coffee cream ice tea chocolate cake flavors donuts shop flavor
     23 R: 0.096 P: 0.096 sandwich burgers sandwiches bacon coffee eggs toast bread brunch pancakes
   3 R: 0.171 P: 0.028 check years music cool asked told free decent room money
     31 R: 0.001 P: 0.001 check asked free years cool manager decent found helpful told
     32 R: 0.007 P: 0.007 

KeyboardInterrupt: 

In [12]:
tf.trainable_variables()

[<tf.Variable 'topic/enc/hidden_bow/kernel:0' shape=(6177, 256) dtype=float32_ref>,
 <tf.Variable 'topic/enc/hidden_bow/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'topic/enc/mean_bow/kernel:0' shape=(256, 32) dtype=float32_ref>,
 <tf.Variable 'topic/enc/mean_bow/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'topic/enc/logvar_bow/kernel:0' shape=(256, 32) dtype=float32_ref>,
 <tf.Variable 'topic/enc/logvar_bow/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'topic/enc/sticks_path/init_state_parent:0' shape=(1, 32) dtype=float32_ref>,
 <tf.Variable 'topic/enc/sticks_path/input/ancestral/kernel:0' shape=(32, 32) dtype=float32_ref>,
 <tf.Variable 'topic/enc/sticks_path/input/ancestral/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'topic/enc/sticks_path/input/fraternal/kernel:0' shape=(32, 32) dtype=float32_ref>,
 <tf.Variable 'topic/enc/sticks_path/input/fraternal/bias:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'topic/enc/sticks_path/output/hidden/ke

# analysis

In [21]:
def get_topic_year(sample_batches):
    probs_topics = []
    years = []
    for i, sample_batch in sample_batches:
        probs_topics_batch = sess.run(model.prob_topic, feed_dict=model.get_feed_dict(sample_batch, mode='test'))
        years_batch = [instance.year for instance in sample_batch]
        probs_topics += [probs_topics_batch]
        years += years_batch
    probs_topics = np.concatenate(probs_topics)
    years = np.array(years)

    topic_years = years.dot(probs_topics) / np.sum(probs_topics, 0)
    topic_year = {model.topic_idxs[i]: year for i, year in enumerate(topic_years)}
    return topic_year

In [22]:
sample_batches = get_batches(instances_train, config.batch_size)
topic_year = get_topic_year(sample_batches)
print_topic_year(sess, model, topic_freq_tokens=topic_freq_tokens, topic_year=topic_year)

0 Avg Year: 2005 level analysis english systems domain lexical linguistic test process lexicon
   1 Avg Year: 2000 grammar structure rules semantic form representation rule lexical type structures
     11 Avg Year: 2006 verb semantic syntactic verbs event relations discourse argument relation arguments
     12 Avg Year: 2007 tree parsing dependency parser grammar trees node parse rules syntactic
   2 Avg Year: 2008 english languages dictionary errors morphological chinese lexical rules corpora pos
     21 Avg Year: 2009 entity sense relations relation semantic entities wordnet senses lexical patterns
     22 Avg Year: 2010 translation english source alignment phrase target languages sentences systems parallel
   3 Avg Year: 2008 models probability training algorithm probabilities search parameters sequence segmentation gram
     31 Avg Year: 2011 features feature training performance learning classifier classification accuracy test class
     32 Avg Year: 2015 models network vector emb